In [1]:
import scrapy
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = ['https://www.stats.gov.cn/zt_18555/ztsj/sjzsj/sjz2007/202303/t20230303_1925009.html']
    def parse(self, response):
        soup = BeautifulSoup(response.text, 'html.parser')
        traffic_accidents = soup.find_all('div', class_='traffic_accident')
        for accident in traffic_accidents:
            pass
def fetch_data_from_api():
    response = requests.get('https://www.stats.gov.cn/zt_18555/ztsj/sjzsj/sjz2007/202303/t20230303_1925009.html')
    data = response.json()
    return data

def import_data_from_file(filename):
    data = pd.read_csv(filename)
    return data

def store_data_to_database(data):
    conn = sqlite3.connect('traffic_accidents.db')
    data.to_sql('traffic_accidents', conn, if_exists='replace')
    conn.close()
    
process = scrapy.crawler.CrawlerProcess()
process.crawl(MySpider)
process.start()

api_data = fetch_data_from_api()
file_data = import_data_from_file('traffic_accidents.csv')
merged_data = pd.concat([api_data, file_data])

store_data_to_database(merged_data)


ModuleNotFoundError: No module named 'scrapy'

In [ ]:
import scrapy
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

class MySpider(scrapy.Spider):
    name = 'myspider'

    # 定义爬取的起始网址
    start_urls = ['https://www.stats.gov.cn/zt_18555/ztsj/sjzsj/sjz2007/202303/t20230303_1925009.html']

    # 定义爬取网页内容的处理函数
    def parse(self, response):
        # 使用BeautifulSoup解析网页内容
        soup = BeautifulSoup(response.text, 'html.parser')
        # 提取交通事故数据
        traffic_accidents = soup.find_all('div', class_='traffic_accident')
        for accident in traffic_accidents:
            # 解析数据并存储到数据库中
            pass

def fetch_data_from_api():
    # 使用API接口获取数据
    response = requests.get('http://example.com/api/traffic_accidents')
    data = response.json()
    # 返回结构化数据
    return data
def import_data_from_file(filename):
    # 从本地文件导入数据
    data = pd.read_csv(filename)
    # 返回数据
    return data
def store_data_to_database(data):
    # 连接数据库
    conn = sqlite3.connect('traffic_accidents.db')
    # 将数据存储到数据库中
    data.to_sql('traffic_accidents', conn, if_exists='replace')
    # 关闭数据库连接
    conn.close()

# 爬取网页数据
process = scrapy.crawler.CrawlerProcess()
process.crawl(MySpider)
process.start()
# 获取API数据
api_data = fetch_data_from_api()
# 导入本地文件数据
file_data = import_data_from_file('traffic_accidents.csv')
# 合并数据
merged_data = pd.concat([api_data, file_data])
# 存储数据到数据库
store_data_to_database(merged_data)


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import sqlite3

# 加载停用词列表
stop_words = set(stopwords.words('english'))
# 读取原始数据
raw_data = pd.read_csv('traffic_accidents.csv')
# 缺失值填充
processed_data = raw_data.fillna(method='ffill')
# 异常值处理
processed_data = processed_data[(processed_data['speed'] >= 0) & (processed_data['speed'] <= 120)]
# 删除重复值
processed_data = processed_data.drop_duplicates()
# 文本数据处理
def text_processing(text):
    tokens = word_tokenize(text)  # 分词
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]  # 去停用词
    return ' '.join(filtered_tokens)
processed_data['text'] = processed_data['text'].apply(text_processing)

# 使用TF-IDF进行文本数据向量化处理
tfidf_vectorizer = TfidfVectorizer()
text_vectors = tfidf_vectorizer.fit_transform(processed_data['text'])
# 将文本向量化后的数据转换为DataFrame
text_df = pd.DataFrame(text_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# 合并文本向量化后的数据与原始数据
processed_data = pd.concat([processed_data, text_df], axis=1)
# 存储预处理后的数据到数据仓库中
conn = sqlite3.connect('traffic_accidents.db')
processed_data.to_sql('processed_data', conn, if_exists='replace')
conn.close()


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mlxtend.frequent_patterns import apriori, association_rules

# 读取预处理后的数据
processed_data = pd.read_csv('processed_data.csv')
# 描述性统计
descriptive_stats = processed_data.describe()
# 相关性分析
correlation_matrix = processed_data.corr()
# 假设检验
t_statistic, p_value = stats.ttest_ind(processed_data['feature1'], processed_data['feature2'])
# 构建预测模型
X = processed_data.drop(['target'], axis=1)
y = processed_data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
# 交叉验证评估模型
cv_scores = cross_val_score(model, X, y, cv=5)
# 输出交叉验证评估结果
print("交叉验证评估结果：", cv_scores)
print("平均准确率：", np.mean(cv_scores))

# 输出混淆矩阵和分类报告
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("混淆矩阵：")
print(conf_matrix)
print("分类报告：")
print(class_report)
# 数据挖掘算法应用（聚类分析）
kmeans = KMeans(n_clusters=3, random_state=42)
processed_data['cluster'] = kmeans.fit_predict(X)
# 数据挖掘算法应用（关联规则挖掘）
frequent_itemsets = apriori(processed_data, min_support=0.3, use_colnames=True)
association_rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
# 输出关联规则挖掘结果
print("关联规则挖掘结果：")
print(association_rules)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# 读取预处理后的数据
processed_data = pd.read_csv('processed_data.csv')
# 描述性统计
descriptive_stats = processed_data.describe()
# 相关性分析
correlation_matrix = processed_data.corr()
# 构建预测模型
X = processed_data.drop(['target'], axis=1)
y = processed_data['target']
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# 绘制散点图
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y, palette='viridis')
plt.title('PCA Scatter Plot')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# 可视化界面
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1('交通事故数据分析系统'),
    html.Div([
        dcc.Graph(id='scatter-plot')
    ]),
    html.Div([
        html.Label('选择特征进行PCA降维:'),
        dcc.Dropdown(
            id='pca-dropdown',
            options=[{'label': col, 'value': col} for col in X.columns],
            value=X.columns[0]
        )
    ])
])
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('pca-dropdown', 'value')]
)
def update_scatter_plot(selected_feature):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X[[selected_feature, 'feature2']])
    scatter_plot = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=y)
    scatter_plot.update_layout(title='PCA Scatter Plot', xaxis_title='Principal Component 1',
                                yaxis_title='Principal Component 2')
    return scatter_plot
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process_data', methods=['POST'])
def process_data():
    # 获取前端传来的数据
    data = request.get_json()
    # 进行数据处理和分析（模拟）
    processed_result = {'result': 'Processed data'}
    # 返回处理结果
    return jsonify(processed_result)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>交通事故数据分析系统</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        /* 自定义样式 */
    </style>
</head>
<body>
    <div class="container">
        <h1 class="mt-5">交通事故数据分析系统</h1>
        <div class="row mt-5">
            <div class="col-md-6">
                <h3>数据处理</h3>
                <form id="data-form">
                    <div class="form-group">
                        <label for="data-input">输入数据:</label>
                        <input type="text" class="form-control" id="data-input">
                    </div>
                    <button type="submit" class="btn btn-primary">处理数据</button>
                </form>
            </div>
            <div class="col-md-6">
                <h3>处理结果</h3>
                <div id="result"></div>
            </div>
        </div>
    </div>

    <script src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
    <script>
        $(document).ready(function() {
            $('#data-form').submit(function(e) {
                e.preventDefault();

                // 获取输入的数据
                var inputData = $('#data-input').val();

                // 发送数据给后端进行处理
                $.ajax({
                    type: 'POST',
                    url: '/process_data',
                    contentType: 'application/json',
                    data: JSON.stringify({ 'data': inputData }),
                    success: function(response) {
                        $('#result').html('<p>' + response.result + '</p>');
                    },
                    error: function() {
                        $('#result').html('<p style="color: red;">处理数据时出错。</p>');
                    }
                });
            });
        });
    </script>
</body>
</html>
